# Send REST requests to the WeatherApp.API running in Aspire

In [3]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [4]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-05dba303720e9a8253e857a1b626a17f-7fc9367f7350c27a-00
Name,Value
Date,"Sun, 23 Nov 2025 21:23:05 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌦️🌈🔆

In [5]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-37ad60947cd767e4703ae1721311b91f-88fcf528dd5d08b0-00
Name,Value
Date,"Sun, 23 Nov 2025 21:23:11 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-11-23T21:23:12.3891498Z"""
requestId,"""585cc286-bd8d-4692-bcdf-abd7d62c70e8"""


## POST Colected Weather Data 🌡️☔💨

In [5]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

cherry-29

In [6]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
X-Request-ID,4f8cfa0b-879d-4e2d-a84a-d4dda9442e89
traceparent,00-6cc096b57872f5c982dc8c962e45489d-c2806b384f543fa0-00
Content-Type,application/json
Content-Length,280
Name,Value
Date,"Wed, 21 Jan 2026 23:38:43 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,76


<div style="display:block; height:10rem;"></div>

## GET Traces 🤖

In [7]:
// Get traces and format nicely
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notification Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
                   
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("http://localhost:8000"), DefaultRequestHeaders = { { "X-Api-Key", "11119999" } } };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 NotificationService-----------> User Notification Sent--------> ✉️ reference: cherry-29 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notification Sent TraceId 6cc096b57872f5c982dc8c962e45489d SpanId 2a91217a85f2797f Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "cherry-29" user-notification-event.timestamp "2026-01-21T23:38:46.9552858+00:00" 1 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 2 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId bbd8ffc77d987ee7 Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 2 2 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId d71f074c017a4e92 Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 3 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId c2bc41ee64503d55 Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 4 WeatherApp.API----------------> Domain Event Insertion--------> LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId da85e439a6453efd Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 5 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId bb3bb99e20d8e80f Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 6 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId 71e12f672a26671b Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 7 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId 3c2aed54e7f13885 Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 8 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 6cc096b57872f5c982dc8c962e45489d SpanId 58cf11430d61a9c4 Tags key value domain-event.streamId "ab4d8aa6-370c-41ae-aad0-1c89fbb09dcf" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 9 WeatherApp.API----------------> Domain Event Insertion--------> WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion Tra